In [534]:
import pyodbc
import pandas as pd
import os
from dotenv import load_dotenv
import datetime
import locale
import functions

# Définir la langue française
locale.setlocale(locale.LC_TIME, "fr_FR.UTF-8")

# Load des var env
load_dotenv()

server = os.getenv('SERVER')
database = os.getenv('DATABASE')
username = os.getenv('USERNAME')
password = os.getenv('PASSWORD')
cnxn = pyodbc.connect('DRIVER={ODBC Driver 18 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

query = "SELECT * FROM dbo.Telephonie;"
df_tel = pd.read_sql(query, cnxn)

query = "SELECT * FROM dbo.Eptica;"
df_ept = pd.read_sql(query, cnxn)

ProgrammingError: ('42000', "[42000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Cannot open server 'timeseriescacf' requested by the login. Client with IP address '37.174.9.73' is not allowed to access the server.  To enable access, use the Windows Azure Management Portal or run sp_set_firewall_rule on the master database to create a firewall rule for this IP address or address range.  It may take up to five minutes for this change to take effect. (40615) (SQLDriverConnect)")

In [ ]:
df_tel['date_appel'] = pd.to_datetime(df_tel['date_appel']) # On encode la column "date_appel" pour l'exploiter en timeseries
df_tel['nombre_entrants_corrige'] = df_tel['nombre_entrants_corrige'].astype(int) # On transforme les float en Int car un demi appel n'existe pas
df_tel.head()

,id,date_appel,entite,famille,nombre_entrants_corrige
0,12,2019-01-02,Entite 1,F2,165
1,13,2019-01-02,Entite 1,F3,54
2,14,2019-01-02,Entite 1,F4,1
3,15,2019-01-02,Entite 1,F5,99
4,16,2019-01-02,Entite 1,F6,1487


In [ ]:
# Nombre d'entités (5)
df_tel['entite'].unique()

array(['Entite 1', 'Entite 2', 'Entite 3', 'Entite 4', 'Entite 5'],
      dtype=object)

In [ ]:
# Nombre de famille (7)
df_tel['famille'].unique()

array(['F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F1'], dtype=object)

In [ ]:
# On définit nos dates comme index
df_tel = df_tel.set_index(df_tel['date_appel'].rename("date")).drop("date_appel", axis=1)

# Renome la columns appels reçus
df_tel.rename(columns={"nombre_entrants_corrige":"nb_recus"}, inplace=True)

In [ ]:
# Ajout d'une column jour
df_tel = df_tel.reset_index()
df_tel['jour'] = df_tel['date'].apply(lambda x: str(datetime.datetime(int(x.year), int(x.month), int(x.day)).strftime("%A")))
df_tel = df_tel.set_index('date')

In [ ]:
import plotly.express as px

In [ ]:
df = df_tel.groupby(pd.Grouper(freq = 'Y')).agg(nb_recus_avg = ('nb_recus' , 'mean')).reset_index()
fig = px.line(df, x="date", y="nb_recus_avg")
fig.show()

In [ ]:
#
# Création de graphique dynamique de toutes les Entités avec toutes les Familles
#


for entite in df_tel.entite.unique():
    df = df_tel.loc[df_tel.entite == entite].groupby([pd.Grouper(freq = 'd'), "famille", "jour"]).agg(nb_recus = ('nb_recus' , 'mean')).reset_index()
    
    fig = px.line(df, x="date", y="nb_recus", color='famille',
              hover_data={
                "date": "|%B %d, %Y",
                "jour" : True,
              },
              title=f"Nombre d'appel pour {entite}")

    fig.update_xaxes(
        dtick="M1",
        tickformat="%b\n%Y",
        ticklabelmode="period")
    fig.show()